In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

/media/hdddisk/anaconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/hdddisk/anaconda3/envs/nlp/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-02-12 04:53:16.275075: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-12 04:53:17.091689: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the foll

In [5]:
# Load the tokenizer and model
tokenizer_name = "bert-base-uncased"
model_name = "/media/hdddisk/bert-classify-smsspam-data/results/checkpoint-5580"  # or your specific model name
tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
model = BertForSequenceClassification.from_pretrained(model_name)
model.eval()  # Put the model in evaluation mode

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [16]:
texts = [
    "Your input text here",
    "Some texts",
    "Great news! You're selected"
]
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

In [17]:
with torch.no_grad():  # Disable gradients for inference
   outputs = model(**inputs)
   logits = outputs.logits  # Get the logits (raw scores before activation)

In [19]:
prob = torch.softmax(logits, dim=1)  # Apply softmax along the class dimension

probs_pos = prob[:, 1].tolist()  # Probability of label 1
probs_neg = prob[:, 0].tolist()  # Probability of label 0

for i, text in enumerate(texts):
    print(f"Text: {text}")
    print(f"Probability of label 1: {probs_pos[i]}")
    print(f"Probability of label 0: {probs_neg[i]}")
    print("---")

Text: Your input text here
Probability of label 1: 0.0072843595407903194
Probability of label 0: 0.9927156567573547
---
Text: Some texts
Probability of label 1: 0.00018757855286821723
Probability of label 0: 0.9998124241828918
---
Text: Great news! You're selected
Probability of label 1: 0.0006559456815011799
Probability of label 0: 0.9993440508842468
---
